In [2]:
import os
os.chdir('/mnt/workspace/lizhekai/AmphionVALLEv2') # change to your amphion root path

In [50]:
# put your cheackpoint file (.bin) in the root path of AmphionVALLEv2
# or use your own pretrained weights
ar_model_path = 'valle_ar_mls_encodec.bin'  # download the file on https://huggingface.co/jiaqili3/vallex/resolve/main/valle_ar_mls_encodec.bin?download=true
nar_model_path = 'valle_nar_mls_encodec.bin'  # download the file on https://huggingface.co/jiaqili3/vallex/resolve/main/valle_nar_mls_encodec.bin?download=true
# ar_model_path = './ckpt/valle_gpt_simple/ar_libritts_dev_clean/checkpoint/train-dev-clean-epoch-0843_step-0156000_loss-0.000000/pytorch_model.bin' 
# nar_model_path = './ckpt/valle_gpt_simple/nar_libritts_dev_clean/checkpoint/epoch-0021_step-0004000_loss-5.246324/pytorch_model.bin'

In [44]:
from models.tts.valle_gpt_simple.valle_inference import ValleInference
# change to device='cuda' to use CUDA GPU for fast inference
# change "use_vocos" to True would give better sound quality
# If you meet problem with network, you could set "use_vocos=False", though would give bad quality
model = ValleInference(use_vocos=False, ar_path=ar_model_path, nar_path=nar_model_path, device='cpu')
# model = ValleInference(use_vocos=False, ar_path=ar_model_path, nar_path=nar_model_path, device='cuda')

In [54]:
# prepare inference data
import librosa
import torch
wav, _ = librosa.load('./egs/tts/valle_gpt_simple/prompt_examples/example.wav', sr=24000)
wav = torch.tensor(wav, dtype=torch.float32)
from IPython.display import Audio
Audio(wav, rate = 24000)

In [26]:
prompt_transcript = 'and keeping eternity before the eyes'
target_transcript = 'It presents a unified framework that is inclusive of diverse generation tasks and models, with the added bonus of being easily extendable for new'
from utils.g2p.g2p import phonemizer_g2p as g2p
prompt_transcript = g2p(prompt_transcript, 'en')[1]
target_transcript = g2p(target_transcript, 'en')[1]


In [27]:
prompt_transcript = torch.tensor(prompt_transcript).long()
target_transcript = torch.tensor(target_transcript).long()
transcript = torch.cat([prompt_transcript, target_transcript], dim=-1)
batch = {
        'speech': wav.unsqueeze(0),
        'phone_ids': transcript.unsqueeze(0),
}

In [55]:
configs = [dict(
    top_p=1.0,
    top_k=10,
    temperature=1.1,
    repeat_penalty=1.15,
    max_length=2000,
    num_beams=1,
)] # model inference hyperparameters
output_wav = model(batch, configs)

In [52]:
output_wav

tensor([[[-0.0418, -0.0410, -0.0413,  ...,  0.0007,  0.0009,  0.0006]]])

In [53]:
print('prompt_transcript : and keeping eternity before the eyes')
print('target_transcript : It presents a unified framework that is inclusive of diverse generation tasks and models, with the added bonus of being easily extendable for new')
Audio(output_wav.squeeze(0), rate = 24000)

prompt_transcript : and keeping eternity before the eyes
target_transcript : It presents a unified framework that is inclusive of diverse generation tasks and models, with the added bonus of being easily extendable for new


In [32]:
import torchaudio
torchaudio.save('out.wav', output_wav.squeeze(0), 24000)